In [ ]:
# Основные библиотеки для работы с данными
import pandas as pd  # Для работы с табличными данными
import numpy as np   # Для численных операций
import time          # Для измерения времени выполнения

# Библиотеки для предобработки данных
from sklearn.preprocessing import OneHotEncoder  # Кодирование категориальных признаков
from sklearn.compose import ColumnTransformer    # Преобразование столбцов
from sklearn.base import BaseEstimator, TransformerMixin  # Создание кастомных трансформеров

# Библиотеки для работы с моделями и оценкой
from sklearn.model_selection import (
    train_test_split,      # Разделение на train/test
    StratifiedKFold,       # Стратифицированная кросс-валидация
    cross_val_score        # Оценка с кросс-валидацией
)
from sklearn.metrics import classification_report  # Метрики классификации

# Библиотеки для работы с дисбалансом классов
from imblearn.pipeline import Pipeline as ImbPipeline  # Пайплайн с поддержкой сэмплинга
from imblearn.under_sampling import RandomUnderSampler  # Устранение дисбаланса

# CatBoost
from catboost import CatBoostClassifier  # Классификатор CatBoost

In [7]:
# 1. Кастомный трансформер для feature engineering
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        df = X.copy()
        
        # Извлечение класса из group_title
        def init_class(group_code):
            for N_class in ['11', '10', '9', '8']:
                if N_class in str(group_code)[:15]:
                    return int(N_class)
            pass
        
        df['feat_class'] = df['group_title'].apply(init_class)
        
        # Извлечение формата группы
        def init_group_format(group_code):
            code_to_format = {
                'mini': 'Мини', 'bl': 'Блендед', 'class': 'Блендед',
                'onl+': 'Стандарт', 'micro': 'Микро', 'onl': 'Мини',
                'maximum': 'Микро', 'k+o': 'Онлайн+Класс', 'o+k': 'Онлайн+Класс'
            }
            group_code = str(group_code)
            for format_code, format_title in code_to_format.items():
                if format_code in group_code:
                    return format_title
            pass
        
        df['feat_group_format'] = df['group_title'].apply(init_group_format)
        
        # Извлечение длительности группы в неделях
        def init_group_length(group_code):
            group_code = str(group_code)
            pos = group_code.rfind('w_', 0, -10)
            if pos == -1:
                pass
            group_length = group_code[pos-2:pos].replace('_', '')
            return int(group_length) if group_length.isdigit() else np.nan
        
        df['feat_group_weeks'] = df['group_title'].apply(init_group_length)
        
        # Обработка пропущенных значений
        df['feat_payment_place'] = df['feat_payment_place'].fillna('ИК')
        num_cols = ['feat_term_perc', 'feat_prep_OpenLessons', 'feat_prep_HW', 'feat_prep_attendence']
        for col in num_cols:
            df[col] = df[col].fillna(df[col].mean())
        
        # Удаление ненужных колонок
        cols_to_drop = ['student_id', 'student_email', 'group_id', 'group_title', 
                       'prep_email', 'date_first_l', 'feat_TK_right', 'feat_prep_SIM']
        df.drop([col for col in cols_to_drop if col in df.columns], axis=1, inplace=True)
        
        return df

# 2. Загрузка данных
def load_data(filepath):
    """Загрузка данных из CSV файла"""
    df = pd.read_csv(filepath, sep=';')
    return df

# 3. Основной пайплайн
def main():
    # Загрузка данных
    df = load_data('Clean_Data.csv')
    
    # Разделение на признаки и целевую переменную
    X = df.drop('target', axis=1)
    y = df['target']
    
    # Определение категориальных и числовых признаков
    categorical_features = ['feat_group_format', 'feat_payment_type', 'feat_payment_place']
    numeric_features = [col for col in X.columns 
                       if col not in categorical_features + ['student_id', 'student_email', 'group_id', 
                                                          'group_title', 'prep_email', 'date_first_l']]
    
    # Создание пайплайна
    pipeline = ImbPipeline([
        # Шаг 1: Feature Engineering
        ('feature_engineering', FeatureEngineer()),
        
        # Шаг 2: Препроцессинг
        ('preprocessor', ColumnTransformer(
            transformers=[
                ('num', 'passthrough', numeric_features),
                ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
            ])),
        
        # Шаг 3: Устранение дисбаланса классов
        ('undersampler', RandomUnderSampler(random_state=42)),
        
        # Шаг 4: Модель
        ('classifier', CatBoostClassifier(
            iterations=290,
            depth=9,
            random_state=42,
            learning_rate=0.04,
            verbose=0,
            task_type='GPU',
            boosting_type='Ordered'
        ))
    ])
    
    # Разделение данных на train/test
    X_under, y_under = RandomUnderSampler().fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_under, y_under, test_size=0.2, random_state=42, stratify=y_under)
    
    # Обучение пайплайна
    start_time = time.time()
    pipeline.fit(X_train, y_train)
    print(f"Время обучения: {time.time() - start_time:.2f} секунд")
    
    # Предсказание на тестовых данных
    
    y_pred = pipeline.predict(X_test)
    print(classification_report(y_test, y_pred))
    
    # Кросс-валидация
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(pipeline, X_under, y_under, cv=skf, scoring='f1')
    print(f"F1-score (кросс-валидация): {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

if __name__ == "__main__":
    main()

Время обучения: 28.09 секунд
              precision    recall  f1-score   support

           0       0.83      0.90      0.86      1809
           1       0.89      0.81      0.85      1809

    accuracy                           0.86      3618
   macro avg       0.86      0.86      0.86      3618
weighted avg       0.86      0.86      0.86      3618

F1-score (кросс-валидация): 0.8549 ± 0.0058
